# Worksheet 08

Name:  Aoife Kenny
UID: U68390518

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.102445334197943, 6.729365867031696, 6.485359045260884, 4.984818841215677, 5.525344245913081, 5.423315119396189, 3.6172747834238703, 4.164922962004214, 4.270983998249237, 5.382608555310965]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal( 8 ,1 , 10).tolist()
print(c2)

[9.447637114891553, 8.436755418102168, 7.5871964391294675, 9.85830716755437, 6.434587160660285, 8.956928901499946, 8.447366028098484, 9.221091511043547, 8.749880113393631, 9.662289488635064]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.382608555310965, 9.662289488635064, 4.270983998249237, 8.749880113393631, 4.164922962004214, 3.6172747834238703, 9.221091511043547, 8.447366028098484, 5.423315119396189, 8.956928901499946]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean = 5, variance = 1 and mean = 8, variance = 1 both with an equally likely chance of being in the data set. 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = c1
print(c1)
c2 = c2
print(c2)
prob_c = []
mean = []
var = []
prob_c += [len(c1) / (len(c1) + len(c2))]
prob_c += [len(c2) / (len(c1) + len(c2))]
mean += [ sum(c1)/len(c1)]
mean +=  [sum(c2)/len(c2)]
var += [sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1)]
var += [sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.102445334197943, 6.729365867031696, 6.485359045260884, 4.984818841215677, 5.525344245913081]
[9.447637114891553, 8.436755418102168, 7.5871964391294675, 9.85830716755437, 6.434587160660285]
P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 5.765466666723856,  mean_2 = 8.352896660067568
var_1 = 0.5108027593721809,  var_2 = 1.547591582927976


/Users/aoifekenny/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The values for the mean are close to the real values, the values for the variance are far from the true values. 

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0]*prob_c[0] / [prob_x])
    prob_c1_x.append( pdf_i[1]*prob_c[1] / [prob_x])

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.382608555310965
probability of observing that point if it came from cluster 0 =  0.589748740493904
probability of observing that point if it came from cluster 1 =  0.04086467826405857
point =  9.662289488635064
probability of observing that point if it came from cluster 0 =  1.7985565714195636e-13
probability of observing that point if it came from cluster 1 =  0.1802216677845054
point =  4.270983998249237
probability of observing that point if it came from cluster 0 =  0.010811100203550066
probability of observing that point if it came from cluster 1 =  0.007954003211651801
point =  8.749880113393631
probability of observing that point if it came from cluster 0 =  3.0209099626299196e-08
probability of observing that point if it came from cluster 1 =  0.24943949259498613
point =  4.164922962004214
probability of observing that point if it came from cluster 0 =  0.005763387719843459
probability of observing that point if it came from cluster 1 =  0.006623110239637197
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = [0.29573024],  P(C_2) = [0.70426976]
mean_1 = [4.96042412],  mean_2 = [7.557784]
var_1 = [1.15889627],  var_2 = [2.17979573]


The means are approximately the same distance from the true means and the variances are further from their true variances. P_CJ is inaccurate as the true values are 0.5 and 0.5.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
for p in data:
    print("point = ", p)
    pdf_x = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_x.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_x[0] + prob_c[1] * pdf_x[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_x[0]*prob_c[0] / [prob_x])
    prob_c1_x.append( pdf_x[1]*prob_c[1] / [prob_x])


point =  5.382608555310965
probability of observing that point if it came from cluster 0 =  2.6058415237495985e-09
probability of observing that point if it came from cluster 1 =  0.2388768072354144
point =  9.662289488635064
probability of observing that point if it came from cluster 0 =  2.6058415237495985e-09
probability of observing that point if it came from cluster 1 =  0.2388768072354144
point =  4.270983998249237
probability of observing that point if it came from cluster 0 =  2.6058415237495985e-09
probability of observing that point if it came from cluster 1 =  0.2388768072354144
point =  8.749880113393631
probability of observing that point if it came from cluster 0 =  2.6058415237495985e-09
probability of observing that point if it came from cluster 1 =  0.2388768072354144
point =  4.164922962004214
probability of observing that point if it came from cluster 0 =  2.6058415237495985e-09
probability of observing that point if it came from cluster 1 =  0.2388768072354144
point

These probabilities are less accurate as we dont have the true means and variances. 

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [16]:
assignments = []
for p in range(len(data)):
    if prob_c0_x[p] > prob_c1_x[p]:
        assignments.append(0)
    else:
        assignments.append(1)
    print(data[p],assignments[p])

5.382608555310965 0
9.662289488635064 1
4.270983998249237 0
8.749880113393631 1
4.164922962004214 1
3.6172747834238703 1
9.221091511043547 1
8.447366028098484 1
5.423315119396189 0
8.956928901499946 1
